In [1]:
import pandas as pd
import numpy as np
import pulp

In [2]:
player_stats = pd.read_csv('players_raw.csv')

In [4]:
player_stats['id'] = player_stats.id.astype(str) + '__' + player_stats['first_name'].str.replace(' ','_') +'__'+ player_stats['second_name'].str.replace(' ','_')
player_stats['team_id'] = 'Team_' + player_stats.team.astype(str)

In [7]:
print(player_stats['id'].nunique())
print(player_stats.shape)

581
(1697, 38)


In [9]:
player_stats = player_stats[['id','total_points','now_cost','position','minutes','team_id']]
player_stats

,id,total_points,now_cost,position,minutes,team_id,GW
0,Eric_Bailly__Man_Utd,0,50,2,0,Team_Man Utd,1
1,Keinan_Davis__Aston_Villa,0,45,4,0,Team_Aston Villa,1
2,Ayotomiwa_Dele-Bashiru__Watford,0,45,3,0,Team_Watford,1
3,James_Ward-Prowse__Southampton,2,65,3,90,Team_Southampton,1
4,Bruno_Miguel_Borges_Fernandes__Man_Utd,20,120,3,90,Team_Man Utd,1
...,...,...,...,...,...,...,...
1692,Wilfred_Ndidi__Leicester,2,50,3,90,Team_Leicester,3
1693,Matt_Ritchie__Newcastle,1,50,2,90,Team_Newcastle,3
1694,Nathan_Redmond__Southampton,1,59,3,32,Team_Southampton,3
1695,Mathew_Ryan__Brighton,0,45,1,0,Team_Brighton,3


In [6]:
# objective:
#     Max (SUM(is_player_in_team*total_points))

# contraints:
#     SUM(is_player_in_team*cost + is_player_in_subs*cost)<=1000
#     SUM(is_player_in_team[where element==Def] + is_player_in_subs[where element==Def])==5,..
#     SUM(is_player_in_team[where element==Def])>=3,..
#     a player should not be in_team and in_sub
#     ForEach(is_player_in_team*minutes>=45) --for getting good subs

In [9]:
model = pulp.LpProblem("Constrained_Value_Maximisation", pulp.LpMaximize)

In [7]:
is_player_in_team = [pulp.LpVariable(x+'__team',lowBound=0, upBound=1, cat='Integer') for x in player_stats['id']]
is_player_in_subs = [pulp.LpVariable(x+'__subs',lowBound=0, upBound=1, cat='Integer') for x in player_stats['id']]
is_player_captain = [pulp.LpVariable(x+'__capt',lowBound=0, upBound=1, cat='Integer') for x in player_stats['id']]

In [10]:
# objective
model += sum((is_player_captain[i] + is_player_in_team[i])*points[i] for i in range(len(is_player_in_team)))

In [11]:
# constraint 1 : cost
model += sum((is_player_in_team[i] + is_player_in_subs[i])*costs[i] for i in range(len(is_player_in_team)))<=1000

In [12]:
# constraint 2 - positional
# GK
model += sum((is_player_in_team[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==1)) == 1
model += sum((is_player_in_subs[i] for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==1)) == 1

# DEF
model += sum((is_player_in_team[i] + is_player_in_subs[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==2)) == 5
model += sum((is_player_in_team[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==2)) <= 5
model += sum((is_player_in_team[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==2)) >= 3
model += sum((is_player_in_subs[i] for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==2)) >= 0
model += sum((is_player_in_subs[i] for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==2)) <= 2

# MID
model += sum((is_player_in_team[i] + is_player_in_subs[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==3)) == 5
model += sum((is_player_in_team[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==3)) <= 5
model += sum((is_player_in_team[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==3)) >= 3
model += sum((is_player_in_subs[i] for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==3)) >= 0
model += sum((is_player_in_subs[i] for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==3)) <= 2

# FWD
model += sum((is_player_in_team[i] + is_player_in_subs[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==4)) == 3
model += sum((is_player_in_team[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==4)) <= 3
model += sum((is_player_in_team[i] for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==4)) >= 1
model += sum((is_player_in_subs[i] for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==4)) >= 0
model += sum((is_player_in_subs[i] for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==4)) <= 2

In [13]:
# constraint 4 - number of captain should be 1
model += sum((is_player_captain[i] for i in range(len(is_player_captain)))) == 1

# constraint 4 - number of team players should be 11
model += sum((is_player_in_team[i] for i in range(len(is_player_in_team)))) == 11

model += sum((is_player_in_subs[i] for i in range(len(is_player_in_subs)))) == 4

In [14]:
for i in range(len(is_player_in_team)):
    model += (is_player_in_team[i]+is_player_in_subs[i])<=1 # in_team or in_sub - not both
    model += (is_player_in_team[i]-is_player_captain[i])>=0 # captain should be in team
    

In [15]:
for club_id in np.unique(player_stats.team_id):
        model += sum(is_player_in_team[i] + is_player_in_subs[i] for i in range(len(is_player_in_team)) if player_stats.team_id.iloc[i] == club_id) <= 3  # max 3 players

In [16]:
# pulp.listSolvers(onlyAvailable=True)
print(model.solve(pulp.PULP_CBC_CMD(msg=True)))
print(model.objective.value())

1
2327.0


In [17]:
# soltution checks

# SUBS
print("SUBS variables' values")
print("SUM :",sum([player.value() for player in is_player_in_subs]))
print("MIN :",min([player.value() for player in is_player_in_subs]))
print("MAX :",max([player.value() for player in is_player_in_subs]))

print()

# TEAM
print("TEAM variables' values")
print("SUM :",sum([player.value() for player in is_player_in_team]))
print("MIN :",min([player.value() for player in is_player_in_team]))
print("MAX :",max([player.value() for player in is_player_in_team]))

print()

# CAPTAIN
print("CAPTAIN variables' values")
print("SUM :",sum([player.value() for player in is_player_captain]))
print("MIN :",min([player.value() for player in is_player_captain]))
print("MAX :",max([player.value() for player in is_player_captain]))


SUBS variables' values
SUM : 4.0
MIN : 0.0
MAX : 1.0

TEAM variables' values
SUM : 11.0
MIN : 0.0
MAX : 1.0

CAPTAIN variables' values
SUM : 1.0
MIN : 0.0
MAX : 1.0


In [19]:
[(i,player.value()) for i,player in enumerate(is_player_in_subs)]

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0),
 (43, 0.0),
 (44, 0.0),
 (45, 0.0),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.0),
 (50, 0.0),
 (51, 0.0),
 (52, 0.0),
 (53, 0.0),
 (54, 0.0),
 (55, 0.0),
 (56, 0.0),
 (57, 0.0),
 (58, 0.0),
 (59, 0.0),
 (60, 0.0),
 (61, 0.0),
 (62, 0.0),
 (63, 0.0),
 (64, 0.0),
 (65, 0.0),
 (66, 0.0),
 (67, 0.0),
 (68, 0.0),
 (69, 0.0),
 (70, 0.0),
 (71, 0.0),
 (72, 0.0),
 (73, 0.0),
 (74, 0.0),
 (75, 0.0),
 (76, 0.0),
 (77, 0.0),
 (78, 0.0),
 (79, 0.0),
 (80, 0.0),
 (81, 0.0),
 (82, 0.0),
 (83, 0.0),
 (

In [20]:
[(i,player.value()) for i,player in enumerate(is_player_in_team)]

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 1.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0),
 (43, 0.0),
 (44, 0.0),
 (45, 0.0),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.0),
 (50, 0.0),
 (51, 0.0),
 (52, 0.0),
 (53, 0.0),
 (54, 0.0),
 (55, 0.0),
 (56, 0.0),
 (57, 0.0),
 (58, 0.0),
 (59, 0.0),
 (60, 0.0),
 (61, 0.0),
 (62, 0.0),
 (63, 0.0),
 (64, 0.0),
 (65, 0.0),
 (66, 0.0),
 (67, 0.0),
 (68, 0.0),
 (69, 0.0),
 (70, 0.0),
 (71, 0.0),
 (72, 0.0),
 (73, 0.0),
 (74, 0.0),
 (75, 0.0),
 (76, 1.0),
 (77, 0.0),
 (78, 0.0),
 (79, 0.0),
 (80, 0.0),
 (81, 0.0),
 (82, 0.0),
 (83, 0.0),
 (

In [21]:
pulp.LpStatus[model.status]

'Optimal'

In [22]:
# result of constraint 1 : cost
sum((is_player_in_team[i].value() + is_player_in_subs[i].value())*costs[i] for i in range(len(is_player_in_team))) #<=1000

1000.0

In [23]:
# constraint 2 - positional
# GK
print('GK in team:',sum((is_player_in_team[i].value() for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==1)))# == 1
print('GK in subs:',sum((is_player_in_subs[i].value() for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==1)))# == 1
print()

# DEF
print('DEF Total:',sum((is_player_in_team[i].value() + is_player_in_subs[i].value() for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==2)))# == 5
print('DEF in team:',sum((is_player_in_team[i].value() for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==2)))# <= 5
print('DEF in subs:',sum((is_player_in_subs[i].value() for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==2)))# >= 0
print()

# MID
print('MID Total:',sum((is_player_in_team[i].value() + is_player_in_subs[i].value() for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==3)))# == 5
print('MID in team:',sum((is_player_in_team[i].value() for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==3)))# <= 5
print('MID in subs:',sum((is_player_in_subs[i].value() for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==3)))# >= 0
print()

# FWD
print('FWD Total:',sum((is_player_in_team[i].value() + is_player_in_subs[i].value() for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==3)))# == 3
print('FWD in team:',sum((is_player_in_team[i].value() for i in range(len(is_player_in_team)) if player_stats['element_type'].iloc[i]==4)))# <= 3
print('FWD in subs:',sum((is_player_in_subs[i].value() for i in range(len(is_player_in_subs)) if player_stats['element_type'].iloc[i]==4)))# >= 0

GK in team: 1.0
GK in subs: 1.0

DEF Total: 5.0
DEF in team: 3.0
DEF in subs: 2.0

MID Total: 5.0
MID in team: 5.0
MID in subs: 0.0

FWD Total: 5.0
FWD in team: 2.0
FWD in subs: 1.0


In [40]:
# constraint 4 - number of captain should be 1
model += sum((is_player_captain[i] for i in range(len(is_player_captain)))) == 1

# constraint 4 - number of captain should be 1
model += sum((is_player_in_team[i] for i in range(len(is_player_in_team)))) == 11

model += sum((is_player_in_subs[i] for i in range(len(is_player_in_subs)))) == 4

In [41]:
for i in range(len(is_player_in_team)):
    model += (is_player_in_team[i]+is_player_in_subs[i])<=1 # in_team or in_sub - not both
    model += (is_player_in_team[i]-is_player_captain[i])>=0 # captain should be in team
    

In [42]:
for club_id in np.unique(player_stats.team_id):
        model += sum(is_player_in_team[i] + is_player_in_subs[i] for i in range(len(is_player_in_team)) if player_stats.team_id.iloc[i] == club_id) <= 3  # max 3 players